# AutoGluon: freMTPL2freq (Regression) 2

Based on: https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-indepth.html

Changed distribution to poisson? Can't find any docs show how to do that or if it's even possible :(

Added early stopping to align with H20 automl with R (by Nate).

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "mean_absolute_error"  # specify your evaluation metric
stopping = "mean_absolute_error"  # specify your stopping metric
label = "ClaimNb" # name of target or label variable

## Import

In [2]:
# Import packages

from autogluon.tabular import TabularDataset
from autogluon.tabular import TabularPredictor

import pandas as pd
import numpy as np

In [3]:
# Import data

train_data = TabularDataset("freMTPL2freq_dataset_train.csv")
test_data = TabularDataset("freMTPL2freq_dataset_test.csv")

In [4]:
train_data.shape

(474765, 12)

In [5]:
test_data.shape

(203226, 12)

## Tidy

(skipped)

## Transform

In [6]:
# remove "IDpol" (should not be a predictor)

train_data = train_data.drop(["IDpol"], axis=1)
test_data = test_data.drop(["IDpol"], axis=1)

In [7]:
y_test = test_data[label]

## Visualize

(skipped)

## Model 1

In [8]:
%%time

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric,
    problem_type="regression"
).fit(
    train_data,
    time_limit=time_limit,
    ag_args_fit={"stopping_metric": stopping}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_153025/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_153025/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    474765
Train Data Columns: 10
Label Column: ClaimNb
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    71750.87 MB
	Train Data (Original)  Memory Usage: 141.07 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatu

CPU times: user 6h 18min 59s, sys: 8min 54s, total: 6h 27min 53s
Wall time: 13min 52s


In [9]:
y_pred = predictor.predict(test_data)

In [10]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -0.039985119318079465
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985119318079465,
    "root_mean_squared_error": -0.21486121610324158,
    "mean_squared_error": -0.046165342185363875,
    "r2": -0.03587427964308243,
    "pearsonr": 0.010003043367767372,
    "median_absolute_error": -1.6990729027714646e-12
}


In [11]:
# predictor.evaluate(test_data)
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/predictor/predictor.py:1420: FutureWarning: Calling `predictor.predict_proba` when problem_type=regression will raise an AssertionError starting in AutoGluon v0.8. Please call `predictor.predict` instead.
  warnings.warn(
Evaluation: mean_absolute_error on test data: -0.039985119318079465
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985119318079465,
    "root_mean_squared_error": -0.21486121610324158,
    "mean_squared_error": -0.046165342185363875,
    "r2": -0.03587427964308243,
    "pearsonr": 0.010003043367767372,
    "median_absolute_error": -1.6990729027714646e-12
}


In [12]:
predictor.leaderboard(test_data, silent=True).T

,0,1,2,3,4,5,6,7,8,9,10,11
model,NeuralNetTorch,WeightedEnsemble_L2,NeuralNetFastAI,KNeighborsDist,LightGBM,LightGBMLarge,CatBoost,XGBoost,LightGBMXT,KNeighborsUnif,ExtraTreesMSE,RandomForestMSE
score_test,-0.039985,-0.039985,-0.067926,-0.072297,-0.072425,-0.072524,-0.072578,-0.07258,-0.072947,-0.073036,-0.075441,-0.077546
score_val,-0.042334,-0.042334,-0.070266,-0.076406,-0.074476,-0.074557,-0.074781,-0.074668,-0.075217,-0.076874,-0.078331,-0.080284
pred_time_test,0.787942,0.793043,1.76794,3.770327,0.215684,0.173082,0.202717,0.720603,0.313186,4.252614,2.581537,2.850831
pred_time_val,0.047817,0.048458,0.048899,0.186971,0.018449,0.01245,0.012127,0.229948,0.017384,0.157511,0.176581,0.154073
fit_time,378.346786,378.752422,354.475113,1.03666,2.924165,3.820284,46.579589,4.183111,6.246871,4.24487,6.542631,14.827232
pred_time_test_marginal,0.787942,0.0051,1.76794,3.770327,0.215684,0.173082,0.202717,0.720603,0.313186,4.252614,2.581537,2.850831
pred_time_val_marginal,0.047817,0.000641,0.048899,0.186971,0.018449,0.01245,0.012127,0.229948,0.017384,0.157511,0.176581,0.154073
fit_time_marginal,378.346786,0.405636,354.475113,1.03666,2.924165,3.820284,46.579589,4.183111,6.246871,4.24487,6.542631,14.827232
stack_level,1,2,1,1,1,1,1,1,1,1,1,1


In [13]:
predictor.leaderboard(extra_info=True, silent=True).T

,0,1,2,3,4,5,6,7,8,9,10,11
model,NeuralNetTorch,WeightedEnsemble_L2,NeuralNetFastAI,LightGBM,LightGBMLarge,XGBoost,CatBoost,LightGBMXT,KNeighborsDist,KNeighborsUnif,ExtraTreesMSE,RandomForestMSE
score_val,-0.042334,-0.042334,-0.070266,-0.074476,-0.074557,-0.074668,-0.074781,-0.075217,-0.076406,-0.076874,-0.078331,-0.080284
pred_time_val,0.047817,0.048458,0.048899,0.018449,0.01245,0.229948,0.012127,0.017384,0.186971,0.157511,0.176581,0.154073
fit_time,378.346786,378.752422,354.475113,2.924165,3.820284,4.183111,46.579589,6.246871,1.03666,4.24487,6.542631,14.827232
pred_time_val_marginal,0.047817,0.000641,0.048899,0.018449,0.01245,0.229948,0.012127,0.017384,0.186971,0.157511,0.176581,0.154073
fit_time_marginal,378.346786,0.405636,354.475113,2.924165,3.820284,4.183111,46.579589,6.246871,1.03666,4.24487,6.542631,14.827232
stack_level,1,2,1,1,1,1,1,1,1,1,1,1
can_infer,True,True,True,True,True,True,True,True,True,True,True,True
fit_order,10,12,8,4,11,9,6,3,2,1,7,5
num_features,10,1,10,10,10,10,10,10,6,6,10,10


In [14]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [15]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	3.51s	= Expected runtime (0.7s per shuffle set)
	1.55s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Area,3.614439e-08,6.779431e-09,0.000142,5,5.010333e-08,2.218544e-08
DrivAge,2.286729e-08,9.596051e-09,0.002986,5,4.262570e-08,3.108892e-09
Density,1.336826e-08,4.709432e-09,0.001578,5,2.306504e-08,3.671474e-09
Region,7.786567e-09,1.737080e-08,0.186451,5,4.355327e-08,-2.798014e-08
BonusMalus,3.333822e-09,1.165090e-08,0.278544,5,2.732318e-08,-2.065554e-08
VehBrand,-1.449882e-09,1.487335e-08,0.580942,5,2.917455e-08,-3.207431e-08
Exposure,-2.235083e-09,4.859108e-09,0.819089,5,7.769888e-09,-1.224005e-08
VehAge,-4.298406e-09,6.368754e-09,0.897121,5,8.814947e-09,-1.741176e-08
VehPower,-7.676738e-09,5.587757e-09,0.981390,5,3.828531e-09,-1.918201e-08
VehGas,-1.364351e-08,6.261727e-09,0.995896,5,-7.505231e-10,-2.653649e-08


## Model 2

In [16]:
%%time

# Can see performance improve if you specify `num_bag_folds = 5-10, num_stack_levels = 1-3` in the call to `fit()`, 
# but this will increase training times and memory/disk usage.
    # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,                                                              
    # hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  
predictor = TabularPredictor(
    label=label, 
    eval_metric=metric,
    problem_type="regression"
).fit(
    train_data,
    time_limit=time_limit,
    num_bag_folds=5, 
    num_bag_sets=1, 
    num_stack_levels=1,
    ag_args_fit={"stopping_metric": stopping}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_154445/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_154445/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    474765
Train Data Columns: 10
Label Column: ClaimNb
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    66192.59 MB
	Train Data (Original)  Memory Usage: 141.07 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatu

Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the 1147.73s of remaining time.
	-0.0386	 = Validation score   (-mean_absolute_error)
	5.86s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 2458.64s ... Best model: "WeightedEnsemble_L3"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230804_154445/")


CPU times: user 2h 55min, sys: 1min 35s, total: 2h 56min 36s
Wall time: 40min 58s


In [17]:
y_pred = predictor.predict(test_data)

In [18]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -0.039985115978311524
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985115978311524,
    "root_mean_squared_error": -0.2148612263219482,
    "mean_squared_error": -0.046165346576571446,
    "r2": -0.03587437817455141,
    "pearsonr": -0.0005451632078364529,
    "median_absolute_error": -9.009491763745103e-10
}


In [19]:
# predictor.evaluate(test_data)
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/predictor/predictor.py:1420: FutureWarning: Calling `predictor.predict_proba` when problem_type=regression will raise an AssertionError starting in AutoGluon v0.8. Please call `predictor.predict` instead.
  warnings.warn(
Evaluation: mean_absolute_error on test data: -0.039985115978311524
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985115978311524,
    "root_mean_squared_error": -0.2148612263219482,
    "mean_squared_error": -0.046165346576571446,
    "r2": -0.03587437817455141,
    "pearsonr": -0.0005451632078364529,
    "median_absolute_error": -9.009491763745103e-10
}


In [20]:
predictor.leaderboard(test_data, silent=True).T

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
model,NeuralNetTorch_BAG_L2,WeightedEnsemble_L3,NeuralNetTorch_BAG_L1,WeightedEnsemble_L2,NeuralNetFastAI_BAG_L1,NeuralNetFastAI_BAG_L2,CatBoost_BAG_L2,LightGBM_BAG_L2,XGBoost_BAG_L2,LightGBMLarge_BAG_L2,...,LightGBM_BAG_L1,CatBoost_BAG_L1,XGBoost_BAG_L1,LightGBMLarge_BAG_L1,LightGBMXT_BAG_L1,KNeighborsUnif_BAG_L1,ExtraTreesMSE_BAG_L2,RandomForestMSE_BAG_L2,ExtraTreesMSE_BAG_L1,RandomForestMSE_BAG_L1
score_test,-0.039985,-0.039985,-0.039985,-0.039985,-0.065238,-0.067169,-0.072125,-0.072163,-0.072198,-0.072199,...,-0.072418,-0.072537,-0.072576,-0.072584,-0.07295,-0.073063,-0.073974,-0.07468,-0.075443,-0.077438
score_val,-0.038583,-0.038583,-0.038583,-0.038583,-0.06452,-0.066485,-0.071163,-0.071117,-0.071174,-0.071182,...,-0.071479,-0.071574,-0.07159,-0.071613,-0.071975,-0.071918,-0.075807,-0.079536,-0.074546,-0.076606
pred_time_test,38.960111,38.965908,4.012768,4.022015,8.501866,40.463715,32.312202,32.835209,36.199736,33.105504,...,1.277005,0.566268,3.874781,1.057524,1.29239,3.845615,35.881189,34.769706,1.440295,1.349471
pred_time_val,68.499902,68.50838,1.997552,2.005009,3.943664,69.260747,65.570153,66.104759,67.538146,66.047303,...,1.404185,0.416872,1.8968,1.264504,1.499236,15.772926,80.000252,85.567236,11.109471,12.268972
fit_time,1631.435174,1637.298829,417.093335,423.057225,322.597575,1265.163535,1035.986907,940.440172,943.074505,942.873158,...,7.323293,141.391071,8.907699,7.199841,6.473971,1.013871,947.454448,1045.842381,6.010746,13.862888
pred_time_test_marginal,7.130693,0.005797,4.012768,0.009247,8.501866,8.634297,0.482784,1.005791,4.370318,1.276086,...,1.277005,0.566268,3.874781,1.057524,1.29239,3.845615,4.051771,2.940288,1.440295,1.349471
pred_time_val_marginal,3.288949,0.008478,1.997552,0.007457,3.943664,4.049794,0.3592,0.893806,2.327193,0.836349,...,1.404185,0.416872,1.8968,1.264504,1.499236,15.772926,14.789299,20.356283,11.109471,12.268972
fit_time_marginal,698.560917,5.863655,417.093335,5.963891,322.597575,332.289277,103.11265,7.565915,10.200248,9.998901,...,7.323293,141.391071,8.907699,7.199841,6.473971,1.013871,14.580191,112.968124,6.010746,13.862888
stack_level,2,3,1,2,1,2,2,2,2,2,...,1,1,1,1,1,1,2,2,1,1


In [21]:
predictor.leaderboard(extra_info=True, silent=True).T

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
model,NeuralNetTorch_BAG_L2,WeightedEnsemble_L3,NeuralNetTorch_BAG_L1,WeightedEnsemble_L2,NeuralNetFastAI_BAG_L1,NeuralNetFastAI_BAG_L2,LightGBM_BAG_L2,LightGBMXT_BAG_L2,CatBoost_BAG_L2,KNeighborsDist_BAG_L1,...,LightGBM_BAG_L1,CatBoost_BAG_L1,XGBoost_BAG_L1,LightGBMLarge_BAG_L1,KNeighborsUnif_BAG_L1,LightGBMXT_BAG_L1,ExtraTreesMSE_BAG_L1,ExtraTreesMSE_BAG_L2,RandomForestMSE_BAG_L1,RandomForestMSE_BAG_L2
score_val,-0.038583,-0.038583,-0.038583,-0.038583,-0.06452,-0.066485,-0.071117,-0.07114,-0.071163,-0.071163,...,-0.071479,-0.071574,-0.07159,-0.071613,-0.071918,-0.071975,-0.074546,-0.075807,-0.076606,-0.079536
pred_time_val,68.499902,68.50838,1.997552,2.005009,3.943664,69.260747,66.104759,69.302358,65.570153,13.636772,...,1.404185,0.416872,1.8968,1.264504,15.772926,1.499236,11.109471,80.000252,12.268972,85.567236
fit_time,1631.435174,1637.298829,417.093335,423.057225,322.597575,1265.163535,940.440172,949.515543,1035.986907,0.999967,...,7.323293,141.391071,8.907699,7.199841,1.013871,6.473971,6.010746,947.454448,13.862888,1045.842381
pred_time_val_marginal,3.288949,0.008478,1.997552,0.007457,3.943664,4.049794,0.893806,4.091404,0.3592,13.636772,...,1.404185,0.416872,1.8968,1.264504,15.772926,1.499236,11.109471,14.789299,12.268972,20.356283
fit_time_marginal,698.560917,5.863655,417.093335,5.963891,322.597575,332.289277,7.565915,16.641286,103.11265,0.999967,...,7.323293,141.391071,8.907699,7.199841,1.013871,6.473971,6.010746,14.580191,13.862888,112.968124
stack_level,2,3,1,2,1,2,2,2,2,1,...,1,1,1,1,1,1,1,2,1,2
can_infer,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
fit_order,20,22,10,12,8,18,14,13,16,2,...,4,6,9,11,1,3,7,17,5,15
num_features,21,1,10,1,10,21,21,21,21,6,...,10,10,10,10,6,10,10,21,10,21


In [22]:
predictor.get_model_best()

'WeightedEnsemble_L3'

In [23]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	242.93s	= Expected runtime (48.59s per shuffle set)
	86.29s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
DrivAge,1.685793e-08,1.685820e-08,0.044506,5,5.156919e-08,-1.785333e-08
BonusMalus,8.284121e-09,9.446976e-09,0.060720,5,2.773558e-08,-1.116733e-08
Exposure,2.152277e-09,1.153540e-08,0.348973,5,2.590382e-08,-2.159927e-08
Area,-8.959616e-10,3.263301e-09,0.713770,5,5.823220e-09,-7.615143e-09
VehBrand,-2.128337e-09,7.043674e-09,0.731847,5,1.237469e-08,-1.663136e-08
Density,-4.062925e-09,1.070142e-08,0.778135,5,1.797145e-08,-2.609730e-08
Region,-4.877259e-09,8.754178e-09,0.859587,5,1.314771e-08,-2.290223e-08
VehGas,-8.062323e-09,8.409896e-09,0.950659,5,9.253770e-09,-2.537842e-08
VehAge,-1.146578e-08,1.860523e-08,0.879868,5,2.684266e-08,-4.977421e-08
VehPower,-1.378347e-08,7.603413e-09,0.992285,5,1.872065e-09,-2.943900e-08


## Model 3

In [24]:
%%time

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric, 
    problem_type="regression"
).fit(
    train_data,
    time_limit=time_limit, 
    auto_stack=True,
    hyperparameters={'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},
    ag_args_fit={"stopping_metric": stopping}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_163017/"
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_163017/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    474765
Train Data Columns: 10
Label Column: ClaimNb
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    61651.74 MB
	Train Data (Original)  Memory Usage: 141.07 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to bo

Fitting model: LightGBM_BAG_L1 ... Training model for up to 2228.99s of the 3429.12s of remaining time.
	Fitting 8 child models (S6F1 - S6F8) | Fitting with ParallelLocalFoldFittingStrategy
	-0.0729	 = Validation score   (-mean_absolute_error)
	19.35s	 = Training   runtime
	1.33s	 = Validation runtime
Repeating k-fold bagging: 7/20
Fitting model: LightGBM_BAG_L1 ... Training model for up to 2219.84s of the 3419.96s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	-0.0729	 = Validation score   (-mean_absolute_error)
	21.72s	 = Training   runtime
	1.53s	 = Validation runtime
Repeating k-fold bagging: 8/20
Fitting model: LightGBM_BAG_L1 ... Training model for up to 2209.64s of the 3409.77s of remaining time.
	Fitting 8 child models (S8F1 - S8F8) | Fitting with ParallelLocalFoldFittingStrategy
	-0.0729	 = Validation score   (-mean_absolute_error)
	24.34s	 = Training   runtime
	1.74s	 = Validation runtime
Repeating k-fold bagging: 9/2

2023-08-04 16:37:57,067	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-08-04 16:37:57,068	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-08-04 16:37:57,069	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-08-04 16:37:57,069	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-08-04 16:37:57,070	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UN

CPU times: user 28.2 s, sys: 9.44 s, total: 37.6 s
Wall time: 10min 50s


In [25]:
y_pred = predictor.predict(test_data)

In [26]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -0.07378392409657801
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.07378392409657801,
    "root_mean_squared_error": -0.20784771207284186,
    "mean_squared_error": -0.04320067141391497,
    "r2": 0.03064805192380371,
    "pearsonr": 0.1921141237966529,
    "median_absolute_error": -0.035157460719347
}


In [27]:
# predictor.evaluate(test_data)
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/predictor/predictor.py:1420: FutureWarning: Calling `predictor.predict_proba` when problem_type=regression will raise an AssertionError starting in AutoGluon v0.8. Please call `predictor.predict` instead.
  warnings.warn(
Evaluation: mean_absolute_error on test data: -0.07378392409657801
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.07378392409657801,
    "root_mean_squared_error": -0.20784771207284186,
    "mean_squared_error": -0.04320067141391497,
    "r2": 0.03064805192380371,
    "pearsonr": 0.1921141237966529,
    "median_absolute_error": -0.035157460719347
}


In [28]:
predictor.leaderboard(test_data, silent=True).T

,0,1,2,3
model,LightGBM_BAG_L2,WeightedEnsemble_L3,LightGBM_BAG_L1,WeightedEnsemble_L2
score_test,-0.073784,-0.073784,-0.07398,-0.07398
score_val,-0.072698,-0.072698,-0.072858,-0.072858
pred_time_test,12.39646,12.402813,5.991113,5.998271
pred_time_val,8.459944,8.466414,4.338911,4.346328
fit_time,221.709553,221.719246,161.803926,161.821846
pred_time_test_marginal,6.405347,0.006353,5.991113,0.007157
pred_time_val_marginal,4.121034,0.006469,4.338911,0.007418
fit_time_marginal,59.905627,0.009693,161.803926,0.017919
stack_level,2,3,1,2


In [29]:
predictor.leaderboard(extra_info=True, silent=True).T

,0,1,2,3
model,LightGBM_BAG_L2,WeightedEnsemble_L3,LightGBM_BAG_L1,WeightedEnsemble_L2
score_val,-0.072698,-0.072698,-0.072858,-0.072858
pred_time_val,8.459944,8.466414,4.338911,4.346328
fit_time,221.709553,221.719246,161.803926,161.821846
pred_time_val_marginal,4.121034,0.006469,4.338911,0.007418
fit_time_marginal,59.905627,0.009693,161.803926,0.017919
stack_level,2,3,1,2
can_infer,True,True,True,True
fit_order,3,4,1,2
num_features,11,1,10,1


In [30]:
predictor.get_model_best()

'WeightedEnsemble_L3'

In [31]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	107.27s	= Expected runtime (21.45s per shuffle set)
	33.18s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
BonusMalus,0.004757,0.000541,0.000020,5,0.005872,0.003643
Exposure,0.002946,0.000592,0.000185,5,0.004165,0.001727
DrivAge,0.000877,0.000091,0.000014,5,0.001064,0.000690
VehBrand,0.000230,0.000046,0.000174,5,0.000324,0.000137
VehAge,0.000222,0.000036,0.000077,5,0.000295,0.000148
Region,0.000154,0.000046,0.000869,5,0.000249,0.000059
Density,0.000134,0.000053,0.002505,5,0.000244,0.000024
VehGas,0.000039,0.000021,0.007491,5,0.000082,-0.000005
VehPower,0.000017,0.000010,0.009769,5,0.000037,-0.000004
Area,-0.000003,0.000009,0.741544,5,0.000016,-0.000021


## Model 4

In [32]:
%%time

presets = ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric,
    problem_type="regression"
).fit(
    train_data,
    time_limit=time_limit,
    presets=presets,
    ag_args_fit={"stopping_metric": stopping}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_164311/"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Presets specified: ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_164311/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    474765
Train Data Columns: 10
Label Column: ClaimNb
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    59274.43 MB
	Train Data (Original)  Memory Usage: 141.07 MB (0.2% of available memory)
	Inferring data type of each feature based on co

	-0.0386	 = Validation score   (-mean_absolute_error)
	6.02s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 2637.99s ... Best model: "WeightedEnsemble_L3"
Automatically performing refit_full as a post-fit operation (due to `.fit(..., refit_full=True)`
Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBMXT_BAG_L1_FULL ...
	2.57s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	2.72s	 = Training   runtime
Fitting model: RandomForestMSE_BAG_L1_FULL | Skipping fit via cloning parent ...
	13.52s	 = T

CPU times: user 3h 59min 13s, sys: 6min 1s, total: 4h 5min 15s
Wall time: 50min 21s


In [33]:
y_pred = predictor.predict(test_data)

In [34]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -0.039985124080001655
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985124080001655,
    "root_mean_squared_error": -0.2148612248759777,
    "mean_squared_error": -0.04616534595520546,
    "r2": -0.035874364232121536,
    "pearsonr": -0.0005947207224915478,
    "median_absolute_error": -5.265974878376767e-10
}


In [35]:
# predictor.evaluate(test_data)
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/predictor/predictor.py:1420: FutureWarning: Calling `predictor.predict_proba` when problem_type=regression will raise an AssertionError starting in AutoGluon v0.8. Please call `predictor.predict` instead.
  warnings.warn(
Evaluation: mean_absolute_error on test data: -0.039985124080001655
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985124080001655,
    "root_mean_squared_error": -0.2148612248759777,
    "mean_squared_error": -0.04616534595520546,
    "r2": -0.035874364232121536,
    "pearsonr": -0.0005947207224915478,
    "median_absolute_error": -5.265974878376767e-10
}


In [36]:
predictor.leaderboard(test_data, silent=True).T

,0,1,2,3,4,5,6,7,8,9
model,NeuralNetTorch_BAG_L2_FULL,NeuralNetTorch_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL,XGBoost_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,RandomForestMSE_BAG_L1_FULL,ExtraTreesMSE_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL
score_test,-0.039985,-0.039985,-0.072449,-0.072554,-0.072579,-0.072585,-0.072935,-0.073796,-0.073871,-0.075979
score_val,None,None,None,None,None,None,None,None,None,None
pred_time_test,6.629875,0.812914,0.249647,0.219221,0.100876,0.800696,0.247096,0.632686,0.567012,1.667473
pred_time_val,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.996933,11.605571,NaN
fit_time,366.329351,74.206996,2.718664,4.190248,43.388057,4.004203,2.569538,13.515506,5.991122,114.451382
pred_time_test_marginal,1.332254,0.812914,0.249647,0.219221,0.100876,0.800696,0.247096,0.632686,0.567012,1.667473
pred_time_val_marginal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.996933,11.605571,NaN
fit_time_marginal,101.293635,74.206996,2.718664,4.190248,43.388057,4.004203,2.569538,13.515506,5.991122,114.451382
stack_level,2,1,1,1,1,1,1,1,1,1


In [37]:
predictor.leaderboard(extra_info=True, silent=True).T

,0,1,2,3,4,5,6,7,8,9
model,RandomForestMSE_BAG_L1_FULL,ExtraTreesMSE_BAG_L1_FULL,XGBoost_BAG_L1_FULL,NeuralNetTorch_BAG_L2_FULL,NeuralNetTorch_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL
score_val,None,None,None,None,None,None,None,None,None,None
pred_time_val,10.996933,11.605571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time,13.515506,5.991122,4.004203,366.329351,74.206996,114.451382,2.718664,2.569538,4.190248,43.388057
pred_time_val_marginal,10.996933,11.605571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time_marginal,13.515506,5.991122,4.004203,101.293635,74.206996,114.451382,2.718664,2.569538,4.190248,43.388057
stack_level,1,1,1,2,1,1,1,1,1,1
can_infer,True,True,True,True,True,True,True,True,True,True
fit_order,3,5,7,10,8,6,2,1,9,4
num_features,10,10,10,19,10,10,10,10,10,10


In [38]:
predictor.get_model_best()

'NeuralNetTorch_BAG_L2_FULL'

In [39]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	49.65s	= Expected runtime (9.93s per shuffle set)
	16.3s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
DrivAge,4.471766e-08,1.381623e-08,0.000967,5,7.316545e-08,1.626986e-08
BonusMalus,1.496160e-08,8.122685e-09,0.007314,5,3.168632e-08,-1.763118e-09
VehAge,2.569706e-09,9.655921e-09,0.291911,5,2.245138e-08,-1.731197e-08
Density,-1.213854e-10,6.785952e-09,0.514994,5,1.385098e-08,-1.409375e-08
Region,-1.892288e-09,1.379936e-08,0.612788,5,2.652079e-08,-3.030537e-08
VehGas,-2.629924e-09,1.069423e-08,0.694173,5,1.938964e-08,-2.464949e-08
VehPower,-4.249046e-09,1.015251e-08,0.798832,5,1.665512e-08,-2.515321e-08
VehBrand,-4.497578e-09,8.998506e-09,0.836839,5,1.403047e-08,-2.302563e-08
Exposure,-5.499111e-09,5.949281e-09,0.946194,5,6.750541e-09,-1.774876e-08
Area,-9.375130e-09,1.660300e-08,0.862342,5,2.481069e-08,-4.356095e-08


## Model 5

In [40]:
%%time

predictor_light = TabularPredictor(
    label=label, 
    eval_metric=metric,
    problem_type="regression"
).fit(
    train_data,
    time_limit=time_limit,
    hyperparameters='very_light',
    ag_args_fit={"stopping_metric": stopping}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_173426/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_173426/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    474765
Train Data Columns: 10
Label Column: ClaimNb
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    54117.39 MB
	Train Data (Original)  Memory Usage: 141.07 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatu

CPU times: user 5h 38min 53s, sys: 8min 15s, total: 5h 47min 9s
Wall time: 12min 32s


In [41]:
y_pred = predictor.predict(test_data)

In [42]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -0.039985124080001655
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985124080001655,
    "root_mean_squared_error": -0.2148612248759777,
    "mean_squared_error": -0.04616534595520546,
    "r2": -0.035874364232121536,
    "pearsonr": -0.0005947207224915478,
    "median_absolute_error": -5.265974878376767e-10
}


In [43]:
# predictor.evaluate(test_data)
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/predictor/predictor.py:1420: FutureWarning: Calling `predictor.predict_proba` when problem_type=regression will raise an AssertionError starting in AutoGluon v0.8. Please call `predictor.predict` instead.
  warnings.warn(
Evaluation: mean_absolute_error on test data: -0.039985124080001655
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985124080001655,
    "root_mean_squared_error": -0.2148612248759777,
    "mean_squared_error": -0.04616534595520546,
    "r2": -0.035874364232121536,
    "pearsonr": -0.0005947207224915478,
    "median_absolute_error": -5.265974878376767e-10
}


In [44]:
predictor.leaderboard(test_data, silent=True).T

,0,1,2,3,4,5,6,7,8,9
model,NeuralNetTorch_BAG_L2_FULL,NeuralNetTorch_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL,XGBoost_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,RandomForestMSE_BAG_L1_FULL,ExtraTreesMSE_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL
score_test,-0.039985,-0.039985,-0.072449,-0.072554,-0.072579,-0.072585,-0.072935,-0.073796,-0.073871,-0.075979
score_val,None,None,None,None,None,None,None,None,None,None
pred_time_test,6.018164,0.780951,0.253956,0.23641,0.108145,0.666712,0.244235,0.438231,0.449711,1.627453
pred_time_val,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.996933,11.605571,NaN
fit_time,366.329351,74.206996,2.718664,4.190248,43.388057,4.004203,2.569538,13.515506,5.991122,114.451382
pred_time_test_marginal,1.212362,0.780951,0.253956,0.23641,0.108145,0.666712,0.244235,0.438231,0.449711,1.627453
pred_time_val_marginal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.996933,11.605571,NaN
fit_time_marginal,101.293635,74.206996,2.718664,4.190248,43.388057,4.004203,2.569538,13.515506,5.991122,114.451382
stack_level,2,1,1,1,1,1,1,1,1,1


In [45]:
predictor.leaderboard(extra_info=True, silent=True).T

,0,1,2,3,4,5,6,7,8,9
model,RandomForestMSE_BAG_L1_FULL,ExtraTreesMSE_BAG_L1_FULL,XGBoost_BAG_L1_FULL,NeuralNetTorch_BAG_L2_FULL,NeuralNetTorch_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL
score_val,None,None,None,None,None,None,None,None,None,None
pred_time_val,10.996933,11.605571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time,13.515506,5.991122,4.004203,366.329351,74.206996,114.451382,2.718664,2.569538,4.190248,43.388057
pred_time_val_marginal,10.996933,11.605571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time_marginal,13.515506,5.991122,4.004203,101.293635,74.206996,114.451382,2.718664,2.569538,4.190248,43.388057
stack_level,1,1,1,2,1,1,1,1,1,1
can_infer,True,True,True,True,True,True,True,True,True,True
fit_order,3,5,7,10,8,6,2,1,9,4
num_features,10,10,10,19,10,10,10,10,10,10


In [46]:
predictor.get_model_best()

'NeuralNetTorch_BAG_L2_FULL'

In [47]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	60.98s	= Expected runtime (12.2s per shuffle set)
	16.2s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
DrivAge,4.471766e-08,1.381623e-08,0.000967,5,7.316545e-08,1.626986e-08
BonusMalus,1.496160e-08,8.122685e-09,0.007314,5,3.168632e-08,-1.763118e-09
VehAge,2.569706e-09,9.655921e-09,0.291911,5,2.245138e-08,-1.731197e-08
Density,-1.213854e-10,6.785952e-09,0.514994,5,1.385098e-08,-1.409375e-08
Region,-1.892288e-09,1.379936e-08,0.612788,5,2.652079e-08,-3.030537e-08
VehGas,-2.629924e-09,1.069423e-08,0.694173,5,1.938964e-08,-2.464949e-08
VehPower,-4.249046e-09,1.015251e-08,0.798832,5,1.665512e-08,-2.515321e-08
VehBrand,-4.497578e-09,8.998506e-09,0.836839,5,1.403047e-08,-2.302563e-08
Exposure,-5.499111e-09,5.949281e-09,0.946194,5,6.750541e-09,-1.774876e-08
Area,-9.375130e-09,1.660300e-08,0.862342,5,2.481069e-08,-4.356095e-08


## Model 6

In [48]:
%%time

excluded_model_types = ['KNN', 'NN', 'custom']

predictor_light = TabularPredictor(
    label=label, 
    eval_metric=metric,
    problem_type="regression"
).fit(
    train_data,
    time_limit=time_limit,
    excluded_model_types=excluded_model_types,
    ag_args_fit={"stopping_metric": stopping}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_174740/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_174740/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    474765
Train Data Columns: 10
Label Column: ClaimNb
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    53712.23 MB
	Train Data (Original)  Memory Usage: 141.07 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatu

CPU times: user 6h 39s, sys: 9min 7s, total: 6h 9min 47s
Wall time: 13min


In [49]:
y_pred = predictor.predict(test_data)

In [50]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: mean_absolute_error on test data: -0.039985124080001655
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985124080001655,
    "root_mean_squared_error": -0.2148612248759777,
    "mean_squared_error": -0.04616534595520546,
    "r2": -0.035874364232121536,
    "pearsonr": -0.0005947207224915478,
    "median_absolute_error": -5.265974878376767e-10
}


In [51]:
# predictor.evaluate(test_data)
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/predictor/predictor.py:1420: FutureWarning: Calling `predictor.predict_proba` when problem_type=regression will raise an AssertionError starting in AutoGluon v0.8. Please call `predictor.predict` instead.
  warnings.warn(
Evaluation: mean_absolute_error on test data: -0.039985124080001655
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -0.039985124080001655,
    "root_mean_squared_error": -0.2148612248759777,
    "mean_squared_error": -0.04616534595520546,
    "r2": -0.035874364232121536,
    "pearsonr": -0.0005947207224915478,
    "median_absolute_error": -5.265974878376767e-10
}


In [52]:
predictor.leaderboard(test_data, silent=True).T

,0,1,2,3,4,5,6,7,8,9
model,NeuralNetTorch_BAG_L2_FULL,NeuralNetTorch_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL,XGBoost_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,RandomForestMSE_BAG_L1_FULL,ExtraTreesMSE_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL
score_test,-0.039985,-0.039985,-0.072449,-0.072554,-0.072579,-0.072585,-0.072935,-0.073796,-0.073871,-0.075979
score_val,None,None,None,None,None,None,None,None,None,None
pred_time_test,6.317364,0.843038,0.242233,0.27229,0.100394,0.592945,0.233836,0.460762,0.650853,1.597183
pred_time_val,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.996933,11.605571,NaN
fit_time,366.329351,74.206996,2.718664,4.190248,43.388057,4.004203,2.569538,13.515506,5.991122,114.451382
pred_time_test_marginal,1.323831,0.843038,0.242233,0.27229,0.100394,0.592945,0.233836,0.460762,0.650853,1.597183
pred_time_val_marginal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.996933,11.605571,NaN
fit_time_marginal,101.293635,74.206996,2.718664,4.190248,43.388057,4.004203,2.569538,13.515506,5.991122,114.451382
stack_level,2,1,1,1,1,1,1,1,1,1


In [53]:
predictor.leaderboard(extra_info=True, silent=True).T

,0,1,2,3,4,5,6,7,8,9
model,RandomForestMSE_BAG_L1_FULL,ExtraTreesMSE_BAG_L1_FULL,XGBoost_BAG_L1_FULL,NeuralNetTorch_BAG_L2_FULL,NeuralNetTorch_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL,LightGBM_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL
score_val,None,None,None,None,None,None,None,None,None,None
pred_time_val,10.996933,11.605571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time,13.515506,5.991122,4.004203,366.329351,74.206996,114.451382,2.718664,2.569538,4.190248,43.388057
pred_time_val_marginal,10.996933,11.605571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time_marginal,13.515506,5.991122,4.004203,101.293635,74.206996,114.451382,2.718664,2.569538,4.190248,43.388057
stack_level,1,1,1,2,1,1,1,1,1,1
can_infer,True,True,True,True,True,True,True,True,True,True
fit_order,3,5,7,10,8,6,2,1,9,4
num_features,10,10,10,19,10,10,10,10,10,10


In [54]:
predictor.get_model_best()

'NeuralNetTorch_BAG_L2_FULL'

In [55]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	50.27s	= Expected runtime (10.05s per shuffle set)
	15.71s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
DrivAge,4.471766e-08,1.381623e-08,0.000967,5,7.316545e-08,1.626986e-08
BonusMalus,1.496160e-08,8.122685e-09,0.007314,5,3.168632e-08,-1.763118e-09
VehAge,2.569706e-09,9.655921e-09,0.291911,5,2.245138e-08,-1.731197e-08
Density,-1.213854e-10,6.785952e-09,0.514994,5,1.385098e-08,-1.409375e-08
Region,-1.892288e-09,1.379936e-08,0.612788,5,2.652079e-08,-3.030537e-08
VehGas,-2.629924e-09,1.069423e-08,0.694173,5,1.938964e-08,-2.464949e-08
VehPower,-4.249046e-09,1.015251e-08,0.798832,5,1.665512e-08,-2.515321e-08
VehBrand,-4.497578e-09,8.998506e-09,0.836839,5,1.403047e-08,-2.302563e-08
Exposure,-5.499111e-09,5.949281e-09,0.946194,5,6.750541e-09,-1.774876e-08
Area,-9.375130e-09,1.660300e-08,0.862342,5,2.481069e-08,-4.356095e-08


## Communicate

(skipped)